In [ ]:
import pandas as pd
from ansys.dyna.core.lib.deck import Deck
from ansys.dyna.core.keywords import keywords
from tabulate import tabulate

# Load the two deck files
deck_ref = Deck()
deck_ref.import_file('Controls_Reference.key')

deck_check = Deck()
deck_check.import_file('01_Master.k')

# Extract all the keywords for each deck
keywords_ref = deck_ref.get()
keywords_check = deck_check.get()

# Function to filter keywords by specific type
def compare_keywords(keywords_ref, keywords_check, keyword_type_prefix):
    # Filter the keywords that match the specified prefix
    filtered_ref = filter_keywords_by_type(keywords_ref, keyword_type_prefix)
    filtered_check = filter_keywords_by_type(keywords_check, keyword_type_prefix)

    # Extract the class names for comparison
    ref_class_names = [kw.__class__.__name__ for kw in filtered_ref]
    check_class_names = [kw.__class__.__name__ for kw in filtered_check]

    # Compare the class names and list the differences
    missing_in_check = [name for name in ref_class_names if name not in check_class_names]
    outside_the_ref = [name for name in check_class_names if name not in ref_class_names]

    # Adjust the lengths of the lists to create a consistent DataFrame
    max_length = max(len(missing_in_check), len(outside_the_ref))
    missing_in_check += [None] * (max_length - len(missing_in_check))
    outside_the_ref += [None] * (max_length - len(outside_the_ref))

    # Create and return the differences DataFrame
    return pd.DataFrame({
        f"{keyword_type_prefix} missing in the comparison file": missing_in_check,
        f"{keyword_type_prefix} outside the reference file": outside_the_ref
    })


# Compare the "Control" keywords
df_control_diff = compare_keywords(keywords_ref, keywords_check, "Control")
df_control_diff_title = "Differences of 'Control' keywords"

# Compare the "DATABASE" keywords
df_database_diff = compare_keywords(keywords_ref, keywords_check, "Database")
df_database_diff_title = "Differences of 'DATABASE' keywords"

# Save the differences to a text file with numbering and bold titles (Markdown-style)
with open('keyword_differences.txt', 'w', encoding='utf-8') as f:
    # Write the "Control" keywords differences with title bolding (Markdown-style)
    f.write(f"**{df_control_diff_title}**\n\n")
    
    # Add numbering and table formatting for Control keywords
    df_control_diff.index += 1  # Start numbering from 1
    f.write(tabulate(df_control_diff, headers='keys', tablefmt='fancy_grid', showindex=True, numalign='center', stralign='center'))
    f.write("\n\n")  # New line after Control table

    # Write the "Database" keywords differences with title bolding (Markdown-style)
    f.write(f"**{df_database_diff_title}**\n\n")
    
    # Add numbering and table formatting for Database keywords
    df_database_diff.index += 1  # Start numbering from 1
    f.write(tabulate(df_database_diff, headers='keys', tablefmt='fancy_grid', showindex=True, numalign='center', stralign='center'))

print("The results have been saved in the file 'keyword_differences.txt'.")
